Module - CIS7017 Dissertation
Student ID - #20275320

## Data collection

In [ ]:
# Import all relevant libraries
import pandas as pd
import requests
from tqdm import tqdm

In [ ]:
# Load the dataset
data = pd.read_csv('C:/dataset/US_Accidents.csv')

In [ ]:
data.head()

In [ ]:
# Filter for rows where the State column is 'UT' for Utah
for column in ['State']:
    print(f"\nValue Counts for {column}:")
    print(data[column].value_counts())
    
utah_data = data[data['State'] == 'UT']

### Integrate Altitude data

In [ ]:
# Function to get elevations for a list of latitudes and longitudes
def get_elevations(latitudes, longitudes):
    # Validate latitudes and longitudes
    valid_latitudes = [str(lat) for lat in latitudes if -90 <= lat <= 90]
    valid_longitudes = [str(lon) for lon in longitudes if -180 <= lon <= 180]
    
    # Ensure we have the same number of valid latitudes and longitudes
    if len(valid_latitudes) != len(valid_longitudes) or not valid_latitudes:
        return [None] * len(latitudes)  # Return None for invalid pairs
    
    url = f"https://api.open-meteo.com/v1/elevation?latitude={','.join(valid_latitudes)}&longitude={','.join(valid_longitudes)}"
    try:
        response = requests.get(url)
        response.raise_for_status()  # an exception for HTTP error codes
        return response.json().get('elevation', [None] * len(latitudes))
    except requests.RequestException as e:
        print(f"Request failed: {e}")
        return [None] * len(latitudes)  # Return None for failed requests

# Splitting the DataFrame into chunks of 100 rows to comply with the API's limitation
chunk_size = 100
altitude_list = []

# Wrap the range function with tqdm to see the progress
for start in tqdm(range(0, utah_data.shape[0], chunk_size), desc='Fetching Altitudes'):
    end = start + chunk_size
    batch = utah_data.iloc[start:end]
    latitudes = batch['Start_Lat'].tolist()
    longitudes = batch['Start_Lng'].tolist()
    
    elevations = get_elevations(latitudes, longitudes)
    altitude_list.extend(elevations)

# Add the altitude information to the DataFrame
utah_data['Altitude'] = altitude_list

In [ ]:
# Run again for failed API requests. 
# TODO:: Merge both snippets

def get_elevations(latitudes, longitudes):
    # Construct the API URL with the given latitudes and longitudes
    url = f"https://api.open-meteo.com/v1/elevation?latitude={','.join(map(str, latitudes))}&longitude={','.join(map(str, longitudes))}"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raises an error for bad responses
        return response.json().get('elevation', [None] * len(latitudes))
    except requests.RequestException as e:
        print(f"API request failed: {e}")
        return [None] * len(latitudes)  # Return None for failed requests

# Filter the DataFrame to rows where Altitude is missing (NaN)
missing_altitude_df = utah_data[pd.isna(utah_data['Altitude'])]

# Initialize an empty list to store the fetched altitudes
fetched_altitudes = []

for start in tqdm(range(0, missing_altitude_df.shape[0], chunk_size), desc='Filling Missing Altitudes'):
    end = start + chunk_size
    batch = missing_altitude_df.iloc[start:end]
    latitudes = batch['Start_Lat'].tolist()
    longitudes = batch['Start_Lng'].tolist()
    
    elevations = get_elevations(latitudes, longitudes)
    fetched_altitudes.extend(elevations)

# Update the original DataFrame with the newly fetched altitudes
for (index, altitude), (_, row) in zip(enumerate(fetched_altitudes), missing_altitude_df.iterrows()):
    if altitude is not None:  # Only update if the API call was successful
        utah_data.at[row.name, 'Altitude'] = altitude


In [ ]:
# Save the draft dataset to a CSV file
utah_data.to_csv('utah_traffic_accidents.csv', index=False)

### Integrate Temperature Variations, Oxygen Levels, UV Radiation, Hazards etc.

In [ ]:
#TODO::Integrate Temperature Variations, Oxygen Levels, UV Radiation, Hazards etc.

## Data cleaning

In [ ]:
# Load the saved dataset
utah_data = pd.read_csv('utah_traffic_accidents.csv')

In [ ]:
utah_data

In [ ]:
# Check for missing values
print("Check for missing values \n")
print(utah_data.isnull().sum())

In [ ]:
# Get value counts for a column
for column in ['Precipitation(in)']:
    print(f"\nValue Counts for {column}:")
    print(utah_data[column].value_counts())

In [ ]:
# Dropping columns with less percentage of data and unnecessary columns

missing = pd.DataFrame(utah_data.isnull().sum()).reset_index()
missing.columns = ['Feature', 'Missing_Percent(%)']
missing['Missing_Percent(%)'] = missing['Missing_Percent(%)'].apply(lambda x: x / utah_data.shape[0] * 100)
missing.loc[missing['Missing_Percent(%)']>0,:]

utah_data = utah_data.drop(columns=['Source', 'End_Lat','End_Lng','Wind_Chill(F)', 'Description', 'Street', 'County', 'Zipcode', 'Timezone', 'Airport_Code', 'Weather_Timestamp', 'Amenity', 'Bump', 'Give_Way', 'No_Exit', 'Roundabout', 'Traffic_Calming', 'Turning_Loop'],axis=1)
utah_data.columns

In [ ]:
# Remove data points with missing values (for insignificant amounts)
utah_data = utah_data.dropna(subset=['Nautical_Twilight', 'Precipitation(in)'])
utah_data=utah_data.dropna(axis=0).reset_index(drop=True)

In [ ]:
# Renaming columns
utah_data = utah_data.rename(columns={'Start_Lat': 'Geo_lat', 'Start_Lng': 'Geo_lng', 'Distance(mi)': 'Distance', 'Temperature(F)': 'Temperature',
                                      'Humidity(%)': 'Humidity', 'Pressure(in)': 'Pressure', 'Visibility(mi)': 'Visibility', 'Wind_Speed(mph)': 'Wind_Speed',
                                      'Precipitation(in)': 'Precipitation'})

In [ ]:
utah_data = utah_data.drop(columns=['State'],axis=1)

In [ ]:
# Drop NA columns
# nan_columns = utah_data.columns[utah_data.isnull().any()].tolist()
# if nan_columns:
#     raise ValueError(f"NaN found in columns: {nan_columns}")
utah_data = utah_data.dropna()

In [ ]:
# Calculate 'Time_Duration' in seconds
utah_data['Start_Time'] = pd.to_datetime(utah_data['Start_Time'])
utah_data['End_Time'] = pd.to_datetime(utah_data['End_Time'])
utah_data['Time_Duration'] = (utah_data['End_Time'] - utah_data['Start_Time']).dt.total_seconds()

In [ ]:
#Fixing fractual seconds in Time columns.

try:
    utah_data['Start_Time'] = pd.to_datetime(utah_data['Start_Time']).dt.floor('S')
    utah_data['End_Time'] = pd.to_datetime(utah_data['End_Time']).dt.floor('S')
except Exception as e:
    print("Error converting dates:", e)
    utah_data['Start_Time'] = pd.to_datetime(utah_data['Start_Time'], errors='coerce').dt.floor('S')
    utah_data['End_Time'] = pd.to_datetime(utah_data['End_Time'], errors='coerce').dt.floor('S')
    problematic_starts = utah_data[utah_data['Start_Time'].isna()]
    problematic_ends = utah_data[utah_data['End_Time'].isna()]
    if not problematic_starts.empty or not problematic_ends.empty:
        print("Problematic Start Times:", problematic_starts)
        print("Problematic End Times:", problematic_ends)

In [ ]:
# Clean Up Categorical Features

# Simplify wind direction
print("Wind Direction: ", utah_data['Wind_Direction'].unique())
utah_data.loc[utah_data['Wind_Direction']=='Calm','Wind_Direction'] = 'CALM'
utah_data.loc[(utah_data['Wind_Direction']=='West')|(utah_data['Wind_Direction']=='WSW')|(utah_data['Wind_Direction']=='WNW'),'Wind_Direction'] = 'W'
utah_data.loc[(utah_data['Wind_Direction']=='South')|(utah_data['Wind_Direction']=='SSW')|(utah_data['Wind_Direction']=='SSE'),'Wind_Direction'] = 'S'
utah_data.loc[(utah_data['Wind_Direction']=='North')|(utah_data['Wind_Direction']=='NNW')|(utah_data['Wind_Direction']=='NNE'),'Wind_Direction'] = 'N'
utah_data.loc[(utah_data['Wind_Direction']=='East')|(utah_data['Wind_Direction']=='ESE')|(utah_data['Wind_Direction']=='ENE'),'Wind_Direction'] = 'E'
utah_data.loc[utah_data['Wind_Direction']=='Variable','Wind_Direction'] = 'VAR'
print("Wind Direction after simplification: ", utah_data['Wind_Direction'].unique())

In [ ]:
# Re-saving the cleaned dataset to a CSV file
utah_data.to_csv('utah_traffic_accidents.csv', index=False)

In [ ]:
df = pd.read_csv('utah_traffic_accidents.csv')

In [ ]:
import pandas as pd

# Assuming 'df' is your DataFrame with a 'Date' column, 'Geo_lat', and 'Geo_lng'
# First, ensure your Date column is in datetime format if not already
df['Start_Time'] = pd.to_datetime(df['Start_Time'])

# Grouping by Date and geographic coordinates
df = df.groupby(['Start_Time', 'Geo_lat', 'Geo_lng']).size().reset_index(name='Count')



In [ ]:
# prepare Date field
# df['Start_Time'] = pd.to_datetime(df['Start_Time'])
# df['Date'] = df['Start_Time'].dt.strftime('%Y-%m-%d')  # Create a temporary 'Date' column
# df['Snowfall'] = pd.NA  # Initialize the Snowfall column

In [ ]:
import pandas as pd
df = pd.read_csv('df_with_snowdata.csv')

In [ ]:
import time
import requests
import pandas as pd
from tqdm import tqdm
df = pd.read_csv('df_with_snowdata.csv')

def fetch_snowfall(latitude, longitude, date):
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "start_date": date,
        "end_date": date,
        "daily": "snowfall_sum",
        "timezone": "America/Denver"
    }
    response = requests.get(url, params=params, timeout=50)
    if response.status_code == 200:
        data = response.json()
        if 'daily' in data and 'snowfall_sum' in data['daily'] and data['daily']['snowfall_sum']:
            return data['daily']['snowfall_sum'][0]
    return None

def update_snowfall(df):
    count_calls = 0
    hourly_limit = 10000
    batch_limit = 600  # Maximum calls per batch
    batch_time = 60   # Sleep time in seconds after a batch

    # Filter DataFrame to only include rows where 'Snowfall' is NA
    filtered_df = df[df['Snowfall'].isna()]

    # Initialize the progress bar
    pbar = tqdm(total=filtered_df.shape[0], desc="Processing snowfall data")

    for date, group in filtered_df.groupby('Date'):
        for idx, row in group.iterrows():
            if count_calls >= batch_limit:
                print("Reached 600 calls, waiting for 60 seconds...")
                time.sleep(batch_time)  # Wait for 60 seconds
                count_calls = 0  # Reset count after waiting

            if count_calls >= hourly_limit:
                print("Reached hourly limit, stopping...")
                pbar.close()  # Ensure to close the progress bar properly
                return  # Stop processing if hourly limit is reached

            snowfall = fetch_snowfall(row['Geo_lat'], row['Geo_lng'], date)
            if snowfall is not None:
                df.at[idx, 'Snowfall'] = snowfall
                count_calls += 1

            pbar.update(1)  # Updating the progress 

    pbar.close()  # Close the progress bar

update_snowfall(df)


In [ ]:
# df['Snowfall'].isnull().values.sum()
# df['Snowfall'].value_counts().sum()
# df['Snowfall'].isna().sum()

# Re-saving the cleaned dataset to a CSV file
# df.to_csv('df_with_snowdata.csv', index=False)
df


In [ ]:
# To populate the remaining NA values in the 'Snowfall' column by using the nearest known values 

import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from sklearn.neighbors import BallTree
import numpy as np

df = pd.read_csv('df_with_snowdata.csv')

# Separate rows with known and unknown snowfall values
known_snowfall = df.dropna(subset=['Snowfall'])
na_snowfall = df[df['Snowfall'].isna()]

# Convert DataFrames to GeoDataFrames
known_snowfall['geometry'] = known_snowfall.apply(lambda row: Point(row['Geo_lng'], row['Geo_lat']), axis=1)
na_snowfall['geometry'] = na_snowfall.apply(lambda row: Point(row['Geo_lng'], row['Geo_lat']), axis=1)
gdf_known = gpd.GeoDataFrame(known_snowfall, geometry='geometry')
gdf_na = gpd.GeoDataFrame(na_snowfall, geometry='geometry')

# Create spatial index
tree = BallTree(gdf_known[['Geo_lat', 'Geo_lng']], metric='haversine')

# Function to find the nearest point with a known snowfall value
def fill_snowfall(row):
    # Find the index of the nearest point with known snowfall, within the same date
    distance, index = tree.query([[np.radians(row['Geo_lat']), np.radians(row['Geo_lng'])]], return_distance=True)
    closest_points = gdf_known.iloc[index[0]]
    # Filter by the same date
    closest_point_same_date = closest_points[closest_points['Date'] == row['Date']]
    
    if not closest_point_same_date.empty:
        # If there are points with the same date, use the closest one
        return closest_point_same_date.iloc[0]['Snowfall']
    else:
        # If there are no points with the same date, use the closest one regardless of the date
        return closest_points.iloc[0]['Snowfall']

# Apply the function to fill NA values
gdf_na['Snowfall'] = gdf_na.apply(fill_snowfall, axis=1)

# Update the original DataFrame
df.update(gdf_na)

df.to_csv('df_with_snowdata.csv', index=False)

In [ ]:
# Merge snowfall data

snowdf = pd.read_csv('df_with_snowdata.csv')
df = pd.read_csv('utah_traffic_accidents.csv')

df['Start_Time'] = pd.to_datetime(df['Start_Time'])
df['Date'] = df['Start_Time'].dt.strftime('%Y-%m-%d')  # Create a temporary 'Date' column
df['Snowfall'] = pd.NA  # Initialize the Snowfall column


df = pd.merge(df, snowdf[['Geo_lng', 'Geo_lat', 'Date', 'Snowfall']], 
              on=['Geo_lng', 'Geo_lat', 'Date'], 
              how='left', 
              suffixes=('', '_new'))

# If the original Snowfall column in 'df' has NA values, update them with the values from 'snowfalldf'
df['Snowfall'] = df['Snowfall'].combine_first(df['Snowfall_new'])

# Drop the temporary 'Snowfall_new' column after the merge
df.drop(columns=['Snowfall_new', 'Date'], inplace=True)

# save
df.to_csv('utah_traffic_accidents.csv', index=False)